# Attempt at something real



In [1]:
from google.colab import drive
drive.mount('/content/drive')

fs_path = '/content/drive/My Drive/Study/Machine Learning/firststeps/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import json
import matplotlib.pyplot as plt
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.utils import Sequence
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import ModelCheckpoint, TensorBoard
import tensorflow as tf

Using TensorFlow backend.


Set up local :

In [3]:
with open(fs_path + 'training_data/training_data_completeVAL30.json', 'r') as f:
    X, Y, X_val, Y_val, Tx = json.load(f)
    
print('training data collected')

m = len(X)
print('m: ', m)
Ty = Tx



training data collected
m:  78518


Convert X and Y to arrays:

In [4]:

X = np.array(X)
Y = np.array(Y)
X_val = np.array(X_val)
Y_val = np.array(Y_val)

print("X dimensions: ", X.shape)
print("Type of X: ", type(X))
print("Type of X[0]: ", type(X[0]))
print("Type of X[0][0]: ", type(X[0][0]))

X dimensions:  (78518, 256)
Type of X:  <class 'numpy.ndarray'>
Type of X[0]:  <class 'numpy.ndarray'>
Type of X[0][0]:  <class 'numpy.int64'>


In [0]:
class DataGenerator(Sequence):
    'Generates data for Keras. This is a subclass of Sequence'
    def __init__(self, data, batch_size=16, dim=256, n_channels=1,
                 n_classes=333, shuffle=True, n_a = 64):
        """Initialization
        Note that data should be a tuple containing (X, Y)
        """
        self.X_data, self. Y_data = data
        self.dim = dim #the dimension of a single example. Should it be (256, 333), the shape of a training example?
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.a0 = np.zeros((batch_size, n_a))
        self.c0 = np.zeros((batch_size, n_a))

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X_data) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        # list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, Y = self.__data_generation(indexes)
        
        return [X, self.a0, self.c0], Y

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, Tx)
        # Initialization
        X = np.empty((self.batch_size, self.dim))
        Y = np.empty((self.batch_size, self.dim)) #I think this is right...? Because I'll use sparse categorical cross entropy.

        # Generate data
        for i, index in enumerate(indexes):
            # Store sample
            X[i,] = self.X_data[index]

            # Store expected output
            Y[i,] = self.Y_data[index]

        return X, list(Y.transpose(1,0))
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.X_data))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

Set up the variables for the model, and the layers

In [7]:
#number of hidden LSTM states
n_a = 256

# no of unique events
n_values = 333

# Some objects we need
reshapor = Reshape((1, n_values))
reshapor2 = Reshape((1, n_a))
LSTM_cell = LSTM(n_a, return_state = True)
LSTM_cell2 = LSTM(n_a, return_state = True, batch_input_shape=(1, n_a))
# LSTM_cell3 = LSTM(n_a, return_state = True)
densor = Dense(n_values, activation='softmax')

training_generator = DataGenerator((X, Y), n_a = n_a)
# validation_generator = DataGenerator((X_val, Y_val), n_a = n_a)
K.tensorflow_backend._get_available_gpus()

W0718 11:10:44.695937 139791366309760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 11:10:44.720152 139791366309760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0718 11:10:44.721141 139791366309760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0718 11:10:44.722108 139791366309760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0718 11:10:45.200664 139791366309760

['/job:localhost/replica:0/task:0/device:GPU:0']

Here is the model:

In [0]:
def musicmodel_LambdaOneHotLayer(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of a sequence in the corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input with a shape 
    X = Input(shape=(Tx,))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    a2 = a0
    c2 = c0

    # Create empty list to append the outputs to while iterating
    outputs = []
    
    # Step 2: Loop
    for t in range(Tx):
        
        # select the "t"th time step from X. 
        x = Lambda(lambda x: x[:,t])(X)
        # This will be a float indicating class. But we need the class represented in one hot fashion:
        x = Lambda(lambda x: tf.one_hot(K.cast(x, dtype='int32'), 333))(x)
        # We then reshape x to be (1, n_values)
        x = reshapor(x)
        # Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        x = reshapor2(a)
        a2, _, c2 = LSTM_cell2(x, initial_state=[a2, c2])
        # Apply densor to the hidden state output of LSTM_Cell
        out = densor(a2)
        # Add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model(inputs=[X,a0,c0],outputs=outputs)
    
    return model

To run the model:

In [0]:
model = musicmodel_LambdaOneHotLayer(Tx, n_a, n_values)

opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

# model.load_weights('weights/model1layer256lstm400epochs.h5')

model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# tb = TensorBoard(log_dir='logs', histogram_freq=0, batch_size=16, write_graph=False, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')
checkpoint = ModelCheckpoint("weights/0718weights.2x256.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=2, save_best_only=True, save_weights_only=True)

model.fit_generator(generator=training_generator,
                    use_multiprocessing=False, epochs=20, max_queue_size=1, callbacks = [checkpoint])

W0718 11:12:25.864980 139791366309760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0718 11:12:41.233690 139791366309760 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20


Here is the inference model:

In [0]:
def music_inference_model(LSTM_cell, densor, n_values, n_a, Ty = 100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, umber of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    ### START CODE HERE ###
    # Step 1: Create an empty list of "outputs" to later store your predicted values (≈1 line)
    outputs = []
    
    # Step 2: Loop over Ty and generate a value at every time step
    for t in range(Ty):
        
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        
        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = densor(a)

        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        outputs.append(out)
        
        # Step 2.D: Select the next value according to "out", and set "x" to be the one-hot representation of the
        #           selected value, which will be passed as the input to LSTM_cell on the next step. We have provided 
        #           the line of code you need to do this. 
        
        x = Lambda(get_max_pred)(x)
        # x = tf.one_hot(np.argmax(out), 333)
        # x = tf.expand_dims(x, axis=-1)
        # x = tf.expand_dims(x, axis=-1)
        
    # Step 3: Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = Model(inputs=[x0,a0,c0],outputs=outputs)
    
    ### END CODE HERE ###
    
    return inference_model
inference_model = music_inference_model(LSTM_cell, densor, n_values, n_a, Ty)

x_initializer = np.zeros((1, 1, 333))
x_initializer[0][0][308] = 1
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

To make predictions:

In [0]:
def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, 256), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    ### START CODE HERE ###
    # Step 1: Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = inference_model.predict([x_initializer,a_initializer,c_initializer])
    # Step 2: Convert "pred" into an np.array() of indices with the maximum probabilities
    #pred is a list of numpy arrays, with probabilities of events at every time step
    indices = np.argmax(pred, axis=-1)
    # Step 3: Convert indices to one-hot vectors, the shape of the results should be (Ty, n_values)
    results = to_categorical(indices, num_classes=n_values)
    ### END CODE HERE ###
    return results, indices

results, indices = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)
print("np.argmax(results[12]) =", np.argmax(results[12]))
print("np.argmax(results[17]) =", np.argmax(results[17]))
print("list(indices[12:18]) =", list(indices[12:18]))

dump_pickle_data((results, indices), '4_epochs_try.pkl')

print(callbacks_list)